# KG Structure
The structure of the KG will consist of 4 types of Nodes: Concept, Topic, Subject and Standar.
The edges between the nodes will be of 5 types: Standard to Subject, Subject to Topic, Topic to Topic, Topic to Concept, Prerequisite Edge.

In [42]:
#We will be user Torch for this one

import numpy as np
import pandas as pd
import torch
from torch import nn, optim    #nn has backpropagation function and optim has optimization function
import os

In [2]:
cd ..

C:\Prasad\MSIITM\Research\Dataset\train\pre_req_dataset


In [43]:
data = pd.read_csv("KaggleDataset/train.csv")

In [44]:
data.head()

,label,pre requisite,concept,pre requisite taxonomy,concept taxonomy,title 12,title 12.1,title 12.2,title 12.3,title 12.4,...,BOW 656,BOW 657,BOW 658,BOW 659,BOW 660,BOW 661,BOW 662,BOW 663,BOW 664,BOW 665
0,1,Collection and Presentation of Data - II,Assumed Mean Method,IX>>Mathematics>>Statistics>>Graphical Represe...,X>>Mathematics>>Statistics>>Cumulative Frequen...,0.423383,3,1.000000,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
1,1,Frequency Polygon,Assumed Mean Method,IX>>Mathematics>>Statistics>>Graphical Represe...,X>>Mathematics>>Statistics>>Cumulative Frequen...,0.454223,2,1.000000,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
2,1,Mean,Assumed Mean Method,VII>>Mathematics>>Data Handling>>Median,X>>Mathematics>>Statistics>>Cumulative Frequen...,0.809664,1,1.000000,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
3,0,Angle Sum Property,Construction of a Similar Triangle,VII>>Mathematics>>The Triangle and its Propert...,X>>Mathematics>>Triangles>>Similar Figures,0.614234,3,1.000000,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
4,0,Inequality Property of Triangle,Construction of a Similar Triangle,VII>>Mathematics>>The Triangle and its Propert...,X>>Mathematics>>Triangles>>Similar Figures,0.739953,2,0.666667,0.666667,0.666667,...,0,0,0,0,0,0,0,0,0,0


## Now let's create node set where we keep our concepts as nodes.

In [45]:
node_concept = set(x for x in data['concept'])
for x in data['pre requisite']:
    node_concept.add(x) 

In [46]:
print(len(node_concept))

909


In [47]:
# We split the taxonomy into lists using lambda
data['pre requisite taxonomy'] = data['pre requisite taxonomy'].apply(lambda x: x.split(">>"))
data['concept taxonomy'] = data['concept taxonomy'].apply(lambda x: x.split(">>"))

In [48]:
# Apply a lambda function to calculate the maximum length of lists in the column
print(data['pre requisite taxonomy'].apply(lambda x: len(x)).max())
print(data['concept taxonomy'].apply(lambda x: len(x)).max())

5
7


# Add standard nodes in the nodes


In [49]:
node_standard = set()
for x in data['pre requisite taxonomy']:
    node_standard.add(x[0])
for x in data['concept taxonomy']:
    node_standard.add(x[0])

# Add subject nodes in the nodes


In [50]:
node_subject = set()
for x in data['pre requisite taxonomy']:
    node_subject.add(x[1])
for x in data['concept taxonomy']:
    node_subject.add(x[1])

# Add topic nodes in the nodes


In [51]:
node_topic = set()
for x in data['pre requisite taxonomy']:
    node_topic.update(x[2:])

for x in data['concept taxonomy']:
    node_topic.update(x[2:])

# Creating entity2id.csv, first row will represent the entity name and second will represent the id. IDs will be alphanumeric ids consisting of 3 digits.

In [52]:
all_entities = list(node_topic | node_subject | node_standard | node_concept)
entity2id = pd.DataFrame(all_entities, columns=['entities'])
entity2id['id'] = range(0,len(entity2id))
entity2id = entity2id[['id', 'entities']]
entity2id.to_csv("entity2id.csv", index = False)

In [53]:
len(all_entities)

1268

In [54]:
# import random
# import string

# # Combine all entity names into a single list
# all_entities = list(node_topic | node_subject | node_standard | node_concept)

# # Shuffle the entity names to randomize the order
# random.shuffle(all_entities)

# # Function to generate a random alphanumeric 3-digit ID
# def generate_id():
#     return ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))

# # Create the "entity2id.txt" file
# with open("entity2id.txt", "w") as file:
#     # Write the number of entities as the first line
#     file.write(f"{len(all_entities)}\n")

#     # Write header
# #     file.write("EntityID\tEntityName\n")

#     # Write entity names and corresponding IDs
#     for entity_name in all_entities:
#         entity_id = generate_id()
#         file.write(f"{entity_name}\t{entity_id}\n")

# print("entity2id.txt file has been created.")


# Now let's create relationship files
Don't run this file again since we will use already created relationships in the files

In [55]:
relationship = ['next standard of', 'is subject of standard', 'is topic of subject', 'is concept of topic', 'is prerequisite of']
relationship2id = pd.DataFrame(relationship, columns=['relationship'])
relationship2id['id'] = range(0,len(relationship2id))
relationship2id = relationship2id[['id', 'relationship']]
relationship2id.to_csv("relationship2id.csv", index = False)

In [56]:
# relationship = ['next standard of', 'is subject of standard', 'is topic of subject', 'is concept of topic', 'is prerequisite of']

# # Function to generate a random alphanumeric 3-digit ID
# def generate_id():
#     return ''.join(random.choices(string.ascii_uppercase + string.digits, k=2))

# # Create the "entity2id.txt" file
# with open("relation2id.txt", "w") as file:
#     # Write the number of entities as the first line
#     file.write(f"{len(relationship)}\n")

#     # Write header
# #     file.write("EntityID\tEntityName\n")

#     # Write entity names and corresponding IDs
#     for relation_name in relationship:
#         relation_id = generate_id()
#         file.write(f"{relation_name}\t{relation_id}\n")

# print("relation2id.txt file has been created.")

In [57]:
entity_data_dict = pd.Series(entity2id['id'].values, index=entity2id['entities']).to_dict()

In [58]:
# # Function to read the "entity2id.txt" file and store it in a dictionary
# def read_entity2id_file(file_path):
#     entity_data = {}

#     with open(file_path, 'r') as file:
#         # Read the first line containing the number of entities
#         num_entities = int(file.readline().strip())

#         # Skip the header line
# #         file.readline()

#         # Read and store the data in the dictionary
#         for _ in range(num_entities):
#             line = file.readline().strip().split('\t')
#             entity_name, entity_id = line[0], line[1]
#             entity_data[entity_name] = entity_id

#     return entity_data

# # Provide the path to your "entity2id.txt" file
# file_path = "entity2id.txt"

# # Call the function to read the file and store data in a dictionary
# entity_data_dict = read_entity2id_file(file_path)

# file_path = "relation2id.txt"
# relation_data_dict = read_entity2id_file(file_path)


# Creating training file

In [59]:
training_data = []
# We add prerequisite relationship to the training data
for index, x in data.iterrows():
    if int(x['label']):
        training_data.append((entity_data_dict[x['pre requisite']],entity_data_dict[x['concept']],4))

In [60]:
# We add 'is subject of standard' and 'is topic of subject relationship'
for index, x in data.iterrows():
        training_data.append((entity_data_dict[x['pre requisite taxonomy'][1]],entity_data_dict[x['pre requisite taxonomy'][0]],1))
        training_data.append((entity_data_dict[x['concept taxonomy'][1]],entity_data_dict[x['concept taxonomy'][0]],1))
        training_data.append((entity_data_dict[x['pre requisite taxonomy'][2]],entity_data_dict[x['pre requisite taxonomy'][1]],2))
        training_data.append((entity_data_dict[x['concept taxonomy'][2]],entity_data_dict[x['concept taxonomy'][1]],2))

In [61]:
# We add 'is next standard of' relationship

#We take the unique standards available in the dataset
stds = set(x[0] for x in data['pre requisite taxonomy'])
for x in data['concept taxonomy']:
    stds.add(x[0])

print(stds)  # we have all the unique standards in stds set

# Lets add them manually since we don't have exact relationship in the dataset, it is the domain knowledge.

training_data.append((entity_data_dict['VII'],entity_data_dict['VI'],0))
training_data.append((entity_data_dict['VIII'],entity_data_dict['VII'],0))
training_data.append((entity_data_dict['IX'],entity_data_dict['VIII'],0))
training_data.append((entity_data_dict['X'],entity_data_dict['IX'],0))
training_data.append((entity_data_dict['XI'],entity_data_dict['X'],0))
training_data.append((entity_data_dict['XII'],entity_data_dict['XI'],0))


{'VII', 'IX', 'XII', 'VI', 'X', 'XI', 'VIII'}


In [62]:
len(training_data)

12307

In [63]:
# We add "is concept of topic"

for index, x in data.iterrows():
    tmp = x['pre requisite taxonomy'][2:]
    for i in range(len(tmp)-1):
        training_data.append((entity_data_dict[tmp[i+1]],entity_data_dict[tmp[i]],4))

In [64]:
len(training_data)

15259

In [65]:
unique_values = set()
for row in training_data:
    unique_values.add(row[0])
    unique_values.add(row[1])

In [66]:
print(len(unique_values))

852


# Export the training file in the CSV file so that we can use it directly without running whole code everytime.

In [67]:
TrainingData = pd.DataFrame(training_data)

TrainingData.to_csv("TrainingData.csv", index=False, header=["Head","Tail","Relation"])
TrainingData.to_csv("TrainingData.txt", index=False, header=False)

#add number of lines to the txt file in the first line

# Now we have our training data ready, Lets create Knowledge Graph.

In [62]:
import torch
print(torch.__version__)

2.1.1+cpu


In [65]:
cd C:\Prasad\MSIITM\Research\Dataset\train\pre_req_dataset\NoteBooks\OpenKE-OpenKE-PyTorch\openke

C:\Prasad\MSIITM\Research\Dataset\train\pre_req_dataset\NoteBooks\OpenKE-OpenKE-PyTorch\openke
